# Set Up Matrix Input

In [1]:
import numpy as np
import time

# Row and Col size of matrix being bulitiplied
row_size = 64
col_size = 64
row_block = 4

# Generate random input matrix
A = np.random.rand(row_size,col_size)
B = np.random.rand(col_size,row_size)
# A = np.full((row_size,col_size),1)
# B = np.full((col_size,row_size),-1)
# A = np.random.uniform(low=0.0625,high=1,size=(row_size,col_size))
# B = np.random.uniform(low=0.0625,high=1,size=(col_size,row_size))

# pos_values = [2**(-i) for i in range(0,4)]
# neg_values = [-2**(-i) for i in range(0,4)]
# values = np.concatenate([neg_values,pos_values,[0]])
# print(values)
# A_val = np.random.choice(values,row_size*col_size)
# A = A_val.reshape(row_size,col_size)
# B_val = np.random.choice(values,row_size*col_size)
# B = B_val.reshape(col_size,row_size)

# A = np.zeros((row_size,col_size))
# A[::2, :] = 1
# A[1::2,:] = -0.5
# B = np.zeros((col_size,row_size))
# B[::2, :] = -0.5
# B[1::2,:] = 0.25

py_out = np.zeros((row_size,row_size))
py_out = np.matmul(A,B)

# print(f"{A}\n{B}\n{py_out}")

# Converters Float to Fixed Point and Vice Versa

In [2]:
def float_to_fp(value, total_bits=16, frac_bits=8, output_format='hex'):
    # Step 1: Check if the value is negative
    if value < 0:
        is_negative = True
        value = -value  # Work with the positive equivalent for conversion
    else:
        is_negative = False

    # Step 2: Multiply the float by 2^frac_bits to scale it
    scaled_value = round(value * (2 ** frac_bits))
    
    # Step 3: Convert the scaled value to a binary string
    # Format the binary string to the required total bits length
    binary_value = bin(scaled_value)[2:].zfill(total_bits)

    # Step 4: Handle the negative case
    if is_negative:
        # Perform two's complement conversion for negative numbers
        # Subtract the binary value from the max value for two's complement
        max_value = (1 << total_bits)  # This is 2^total_bits
        binary_value = bin(max_value - scaled_value)[2:].zfill(total_bits)

    # Step 5: Convert the binary value to hexadecimal
    hex_value = hex(int(binary_value, 2))[2:].zfill(total_bits // 4)  # Convert to hex and remove '0x'

    # Step 6: Format output based on the `output_format` argument
    if output_format == 'binary':
        return binary_value
    elif output_format == 'hex':
        return hex_value
    elif output_format == 'both':
        return (f"{total_bits}'b{binary_value}", f"{total_bits}'h{hex_value}")
    else:
        raise ValueError("Invalid output format. Choose 'binary', 'hex', or 'both'.")
    
def fp_to_float(value,total_bits=16, frac_bits=8, input_format='hex'):
    if input_format == 'binary': fp_val = int(value,2)
    elif input_format == 'hex': fp_val = int(value,16)
    else: raise ValueError("Invalid input format. Must be 'binary' or 'hex'")
    # Calculate the scaling factor: 2^frac_width
    scaling_factor = 2 ** frac_bits
    # Handle two's complement if the sign bit is set (for negative numbers)
    sign_bit_mask = 1 << (total_bits - 1)  # Mask to isolate the sign bit
    
    # Check if the value is negative by examining the sign bit
    if fp_val & sign_bit_mask:  # If sign bit is 1, it's negative
        # Convert from two's complement
        fp_val = fp_val - (1 << total_bits)  # Apply the two's complement adjustment
    
    # Convert to float by dividing by the scaling factor
    float_value = fp_val / scaling_factor
    
    return float_value

# Example usage:
value = 64  # Example float number

num1 = float_to_fp(value)
inv_num1 = fp_to_float(str(num1))
print(num1,inv_num1)

4000 64.0


# Create File for RTL Testing

In [3]:
def wr_input(matrix,filename):
    # Matrix formatting
    new_row = row_block + 32//4 - 1 + row_block - 1 +1
    new_col = row_block
    submatrix = []
    for i in range(0, col_size, 8):
        temp = matrix[i:i+8,:]
        submatrix.append(temp)
    sys_format = np.zeros((120,new_col))
    temp_format = np.zeros((8,15,4))

    for sub in range(8):
        for i in range(8):
            for j in range(4):
                temp_format[sub][i+j,j] = submatrix[sub][i,j]

    sys_format[:120, :] = temp_format.reshape(120, 4)

    with open(filename, 'w') as f:
        for row in sys_format:
            data = [float_to_fp(i) for i in row]
            # print(data)
            f.write(''.join(data) + '\n')
    # print(f'Formatted Systolic matrix:\n{sys_format}')

# print(f"Data North:\n{A.transpose()}")
# wr_input(B,"..\\testing.txt")

In [4]:
import os

# Create blocks from the big matrix
A_small = [A[i:i+4, :] for i in range(0, row_size, 4)]
B_small = [B[:, i:i+4] for i in range(0, row_size, 4)]

rtl_out = np.zeros((row_size,row_size))

for i in range(row_size//4):
    for j in range(row_size//4):
        wr_input(A_small[i].transpose(),"..\\data_west.txt")
        wr_input(B_small[j],"..\\data_north.txt")
        command = f'cd .. & iverilog.exe -c .\\block_4x32.txt & vvp .\\a.out'
        os.system(command)

        out_temp = []
        with open('..\\out_val.txt', 'r') as f:
            # Iterate through each line in the file
            for line in f:
                temp_data = [line[i:i+4]for i in range(0, len(line.strip()), 4)]
                # print(temp_data)
                line_data = [fp_to_float(line[i:i+4]) for i in range(0, len(line.strip()), 4)]
                out_temp.append(line_data)

        # out_temp = np.array(out_temp)
        # print(out_temp)
        print(i,j)
        rtl_out[4*i:4*(i+1),4*j:4*(j+1)] = out_temp
        err = py_out - rtl_out
        # print(err)


# print(f"Python:\n{py_out}\nRTL:\n{rtl_out}\nError:\n{err}")
# print(err)
print(f'Maximum absolute error: {np.max(np.abs(err))}')

0 0
0 1
1 0
1 1
Python:
[[15.3454056  13.80662928 13.58630577 16.50478095 16.06933263 14.51523321
  16.2399141  17.0646319 ]
 [14.58034058 13.59232397 11.8888125  15.639434   15.63247424 14.58621629
  15.04882875 18.20250696]
 [16.77915324 15.54346751 14.72661273 16.14742881 16.82912084 16.40223151
  17.28218652 18.73452241]
 [15.38850364 15.28531201 14.09253921 16.19245334 16.37225243 15.24788787
  15.42835979 17.45813754]
 [15.41387874 14.72307587 12.95429716 15.26131719 15.59368796 14.06999011
  15.69707729 17.8800221 ]
 [17.9778501  16.05119868 15.26052225 17.0688642  18.07475746 16.12294801
  16.58295179 20.31651203]
 [14.17278111 12.6679063  11.8427262  14.07689713 14.28835705 13.25853601
  14.02905965 16.4621794 ]
 [14.80660873 14.67876699 12.40806828 13.44223825 15.0825402  13.86535649
  14.02316656 15.93288777]]
RTL:
[[15.21875    13.6875     13.46875    16.37890625 15.94921875 14.39453125
  16.11328125 16.95703125]
 [14.46484375 13.46484375 11.77734375 15.5        15.51953125